In [1]:
pip install gradio langchain transformers faiss-gpu sentence-transformers langchain-community wikipedia -U bitsandbytes

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 38.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 104.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import json
import gradio as gr
import wikipedia
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# === Load JSON and prepare LangChain documents ===
with open("/kaggle/input/plant-dieases-llm/plant_disease_rag_json_data_cmpltd.json", "r") as f:
    data = json.load(f)

docs = [
    Document(
        page_content=f"Disease: {item['disease']}\n"
                     f"Symptoms: {item['symptoms']}\n"
                     f"Causes: {item['causes']}\n"
                     f"Treatment: {item['treatment']}",
        metadata={"disease": item["disease"]}
    )
    for item in data
]

# === Split, Embed, and Create Vectorstore ===
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)
split_docs = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
db = FAISS.from_documents(split_docs, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

from huggingface_hub import login
login("")  # paste your token here

# === Load Mistral-7B-Instruct with Quantization ===
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# === Custom Prompt ===
custom_prompt = PromptTemplate.from_template("""
You are an expert assistant in diagnosing and explaining plant diseases.

Instructions:
- Always respond with detailed, plant-science-based explanations.
- Include causes, symptoms, and treatment plans if relevant.
- Be concise but informative.
- When unsure, say "I don't know" rather than guessing.

Previous conversation:
{chat_history}

Relevant context:
{context}

Answer the user's question: {question}
""")

# === Memory to track conversation ===
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question",
    output_key="answer",
    return_messages=True
)

# === Retrieval Chain ===
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt}
)

# === Wikipedia Fallback ===
def fallback_wikipedia_search(query):
    try:
        summary = wikipedia.summary(query, sentences=2)
        return f"(From Wikipedia)\n{summary}"
    except Exception:
        return "Sorry, I couldn't find a good explanation for that."

# === Short / Poor Answer Filter ===
def is_bad_response(response: str, question: str) -> bool:
    return not response or \
        len(response.strip()) < 30 or \
        response.strip().lower() in question.strip().lower() or \
        any(bad in response.lower() for bad in [
            "i don't know", "no information", "not sure", "sorry", "unable to find", "defoliated"
        ])

# === Build recent context string ===
def get_recent_context(history, n=2):
    return "\n".join([f"User: {u}\nAssistant: {a}" for u, a in history[-n:]])

# === Fallback Answer Generator for Causal LLM ===
def fallback_llm_answer(message, history, rag_context=None):
    context_snippet = rag_context or "No RAG context was found."
    chat_context = get_recent_context(history, n=3)

    prompt = f"""
You are an expert plant disease assistant. A user is asking about a plant disease or a follow-up question.

Context from plant disease data or Wikipedia:
{context_snippet}

Conversation so far:
{chat_context}

User: {message}
Assistant:"""

    generated = text_gen_pipeline(prompt)[0]['generated_text']
    return generated.strip()

# === Main Chat Function ===
def chat(message, history):
    try:
        response_dict = qa_chain.invoke({"question": message})
        response = response_dict.get("answer", "") if isinstance(response_dict, dict) else response_dict

        if is_bad_response(response, message):
            docs = retriever.get_relevant_documents(message)
            rag_context = "\n\n".join([doc.page_content for doc in docs]) if docs else ""

            if len(rag_context.strip()) < 100:
                rag_context += "\n\n" + fallback_wikipedia_search(message)

            return fallback_llm_answer(message, history, rag_context=rag_context)

        return response
    except Exception as e:
        return f"An error occurred: {str(e)}"

# === Gradio Chat Interface ===
demo = gr.ChatInterface(
    fn=chat,
    title="🌿 Plant Disease Assistant (Mistral-powered)",
    description="Ask about plant diseases, symptoms, causes, and treatments."
)

demo.launch(share=True)

<ipython-input-2-d87d23387868>:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-2-d87d23387868>:59: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_gen_pipeline)
<ipython-input-2-d87d23387868>:81: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a440297395e799d0da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
